<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import mediapipe as mp
import numpy as np
import cv2
import pandas as pd
from tqdm import tqdm
import gc
from operator import add

2023-08-07 15:31:36.584541: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 15:31:49.478717: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [3]:
cap = cv2.VideoCapture("file://media/kristian/HDD/ASL_Citizen/ASL_Citizen/videos/0023159093449660606-CAFETERIA.mp4")
#cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=2) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        # face
        mp_drawing.draw_landmarks(
            frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(0, 255, 255),
                                   thickness=1,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2))

        # Left hand
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Right Hand
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Posture
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))
        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
results

mediapipe.python.solution_base.SolutionOutputs

In [5]:
len(results.pose_landmarks.landmark)

33

In [6]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [7]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [9]:
result_test = extract_keypoints(results)

In [10]:
result_test

array([ 0.52970541,  0.45343369, -0.51835597, ...,  0.        ,
        0.        ,  0.        ])

In [11]:
def signvideodataframe(filepath):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(filepath)

    # Define the path to the videos directory
    path = 'file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/videos/'

    # Create a 'Path' column by concatenating the path with 'Video file' column
    df['Path'] = path + df['Video file']
    
#    df['Gloss'] = df['Gloss'].str.replace('\d+', '',regex=True)

    # Calculate the frequency of each gloss and create a 'frequency' column
    df['Frequency'] = df['Gloss'].map(df['Gloss'].value_counts())

    # Sort the DataFrame by the 'Gloss' column
    df = df.sort_values(by='Gloss')

    # Reset the index of the DataFrame
    df = df.reset_index(drop=True)

    return df

In [12]:
def frames_from_file(filepath):
    cap = cv2.VideoCapture(filepath)
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    length = round(frames / fps)
    return frames, fps, length

In [13]:
traindf = signvideodataframe('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/splits/train.csv')

In [14]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P31,3827306090663467-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
1,P37,16792698524451422-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
2,P11,6868778695018762-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
3,P11,6870709051348651-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
4,P50,0719792557216079-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [15]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
40149,P37,9716493262876276-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40150,P31,7550572181460327-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40151,P46,47985881750082227-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40152,P50,04671245574824856-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40153,P51,19959052532136146-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [16]:
traindf['Path'][0]

'file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/videos/3827306090663467-1 DOLLAR.mp4'

In [17]:
cap = cv2.VideoCapture(traindf['Path'][0])
#cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=2) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        # face
        mp_drawing.draw_landmarks(
            frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(0, 255, 255),
                                   thickness=1,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2))

        # Left hand
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Right Hand
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Posture
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))
        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [18]:
traindf.sample(20)

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
16970,P29,7590669547710722-HITCHHIKE.mp4,HITCHHIKE,J_02_034,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
24195,P27,21203636481513954-OR.mp4,OR,F_03_081,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
35310,P14,46737223048953713-TENT.mp4,TENT,E_02_012,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16
38248,P50,6046104101537046-WANT 2.mp4,WANT1,E_01_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16
35501,P43,07598140885303795-THERMOMETER.mp4,THERMOMETER,D_02_012,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16
12975,P27,41565812261435275-FINGERSPELLING.mp4,FINGERSPELLING,B_03_065,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
5425,P14,5495199667315276-CASTLE 2.mp4,CASTLE2,K_01_017,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16
39115,P33,28913897920899023-WHY.mp4,WHY,D_01_067,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,14
17590,P27,47262000245136493-HUNGRY.mp4,HUNGRY,C_01_010,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
26065,P29,37115423300976147-PIZZA CUTTER.mp4,PIZZACUTTER,E_02_084,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16


In [19]:
traindf = traindf[traindf['Frequency'] > 17]
traindf = traindf.reset_index(drop=True)

In [20]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P52,07157565148825373-seedAXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
1,P28,7179300005186042-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
2,P29,16216064841959765-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
3,P37,6193814382865199-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
4,P40,5947453960317015-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19


In [21]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
945,P50,9161417844146778-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
946,P14,00930662603221255-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
947,P27,82063651021682-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
948,P40,5268072837528903-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
949,P40,6363286086951516-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20


In [22]:
traindf.describe()

,Frequency
count,950.000000
mean,19.082105
std,1.316026
min,18.000000
25%,18.000000
50%,19.000000
75%,20.000000
max,24.000000


In [23]:
#traindf['Gloss'] = traindf['Gloss'].str.replace('\d+', '',regex=True)

In [24]:
traindf['Gloss'].nunique()

50

In [25]:
words = traindf['Gloss'].unique()
words

array(['AXE1', 'BACKPACK1', 'BASKETBALL1', 'BEE1', 'BELT1', 'BITE1',
       'BREAKFAST1', 'CANCER1', 'CHRISTMAS1', 'CONFUSED1', 'DARK1',
       'DEAF1', 'DECIDE1', 'DEMAND1', 'DEVELOP1', 'DINNER1', 'DOG1',
       'DOWNSIZE1', 'DRAG1', 'EAT1', 'EDIT1', 'ELEVATOR1', 'FINE1',
       'FLOAT1', 'FOREIGNER1', 'GUESS1', 'HALLOWEEN1', 'HOSPITAL1',
       'HURDLE/TRIP1', 'JEWELRY', 'KNIGHT1', 'LOCK1', 'LUNCH1', 'MAPLE',
       'MEAT1', 'MECHANIC1', 'MICROSCOPE1', 'MOVIE1', 'NOON1', 'PARTY1',
       'PATIENT2', 'RAZOR2', 'RIVER1', 'ROCKINGCHAIR1', 'SHAVE1', 'SINK',
       'SQUEEZE', 'THEY1', 'TWINS1', 'WHATFOR1'], dtype=object)

In [26]:
valdf = signvideodataframe('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/splits/val.csv') 

In [27]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P26,22595012150860327-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
1,P39,7421622940519235-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
2,P21,686738356933241-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
3,P12,9219095671540121-5 DOLLARS.mp4,5DOLLARS,B_01_062,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
4,P21,1448188216215387-5 DOLLARS.mp4,5DOLLARS,B_01_062,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4


In [28]:
valdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
10299,P21,6959326205750493-ZOOM IN.mp4,ZOOMIN,B_02_056,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
10300,P12,7758716133684984-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
10301,P39,844134294032034-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
10302,P21,5548062993721732-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
10303,P26,19366754134806952-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4


In [29]:
testdf = signvideodataframe('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/splits/test.csv')

In [30]:
testdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P18,23521769221811684-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
1,P42,023931338852502426-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
2,P49,4893817008748198-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
3,P17,13991818149960333-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
4,P49,34625615110480457-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13


In [31]:
testdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
32936,P18,4320702510886756-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32937,P9,7676354653247301-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32938,P47,5386272465310649-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32939,P18,738440364224181-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32940,P17,9953298353288469-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13


In [32]:
valdf = valdf[valdf['Gloss'].isin(words)]
valdf = valdf.reset_index(drop=True)

In [33]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P39,19778675091674147-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
1,P26,8581142177964065-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
2,P39,3877478645046861-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
3,P21,8521417940364975-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
4,P5,9990244640190733-BACKPACK.mp4,BACKPACK1,G_03_091,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3


In [34]:
valdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
186,P26,032677896012150764-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
187,P5,6523145816470133-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
188,P39,0283886564670357-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
189,P21,6142521746642153-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
190,P12,06408604416165864-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5


In [35]:
valdf['Gloss'].nunique()

50

In [36]:
valdf.describe()

,Frequency
count,191.000000
mean,3.963351
std,0.770316
min,3.000000
25%,3.000000
50%,4.000000
75%,4.000000
max,6.000000


In [37]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P39,19778675091674147-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
1,P26,8581142177964065-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
2,P39,3877478645046861-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
3,P21,8521417940364975-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
4,P5,9990244640190733-BACKPACK.mp4,BACKPACK1,G_03_091,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3


In [38]:
testdf = testdf[testdf['Gloss'].isin(words)]
testdf = testdf.reset_index(drop=True)

In [39]:
testdf['Gloss'].nunique()

50

In [40]:
testdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P15,33721516025652254-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
1,P47,519067006979435-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
2,P35,5104381603195376-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
3,P42,44458614013793873-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
4,P42,9604797909481075-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [41]:
testdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
724,P42,3448123355846451-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
725,P47,034046510887022485-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
726,P22,9359050586202402-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
727,P42,2748457214117681-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
728,P15,540014801916062-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18


In [42]:
testdf.describe()

,Frequency
count,729.000000
mean,14.978052
std,2.229192
min,9.000000
25%,14.000000
50%,15.000000
75%,16.000000
max,20.000000


In [43]:
traindf['Gloss'] = traindf['Gloss'].str.replace('\d+', '',regex=True)
valdf['Gloss'] = valdf['Gloss'].str.replace('\d+', '',regex=True)
testdf['Gloss'] = testdf['Gloss'].str.replace('\d+', '',regex=True)

In [44]:
words = traindf['Gloss'].unique()
words

array(['AXE', 'BACKPACK', 'BASKETBALL', 'BEE', 'BELT', 'BITE',
       'BREAKFAST', 'CANCER', 'CHRISTMAS', 'CONFUSED', 'DARK', 'DEAF',
       'DECIDE', 'DEMAND', 'DEVELOP', 'DINNER', 'DOG', 'DOWNSIZE', 'DRAG',
       'EAT', 'EDIT', 'ELEVATOR', 'FINE', 'FLOAT', 'FOREIGNER', 'GUESS',
       'HALLOWEEN', 'HOSPITAL', 'HURDLE/TRIP', 'JEWELRY', 'KNIGHT',
       'LOCK', 'LUNCH', 'MAPLE', 'MEAT', 'MECHANIC', 'MICROSCOPE',
       'MOVIE', 'NOON', 'PARTY', 'PATIENT', 'RAZOR', 'RIVER',
       'ROCKINGCHAIR', 'SHAVE', 'SINK', 'SQUEEZE', 'THEY', 'TWINS',
       'WHATFOR'], dtype=object)

In [45]:
words = valdf['Gloss'].unique()
words

array(['AXE', 'BACKPACK', 'BASKETBALL', 'BEE', 'BELT', 'BITE',
       'BREAKFAST', 'CANCER', 'CHRISTMAS', 'CONFUSED', 'DARK', 'DEAF',
       'DECIDE', 'DEMAND', 'DEVELOP', 'DINNER', 'DOG', 'DOWNSIZE', 'DRAG',
       'EAT', 'EDIT', 'ELEVATOR', 'FINE', 'FLOAT', 'FOREIGNER', 'GUESS',
       'HALLOWEEN', 'HOSPITAL', 'HURDLE/TRIP', 'JEWELRY', 'KNIGHT',
       'LOCK', 'LUNCH', 'MAPLE', 'MEAT', 'MECHANIC', 'MICROSCOPE',
       'MOVIE', 'NOON', 'PARTY', 'PATIENT', 'RAZOR', 'RIVER',
       'ROCKINGCHAIR', 'SHAVE', 'SINK', 'SQUEEZE', 'THEY', 'TWINS',
       'WHATFOR'], dtype=object)

In [46]:
words = testdf['Gloss'].unique()
words

array(['AXE', 'BACKPACK', 'BASKETBALL', 'BEE', 'BELT', 'BITE',
       'BREAKFAST', 'CANCER', 'CHRISTMAS', 'CONFUSED', 'DARK', 'DEAF',
       'DECIDE', 'DEMAND', 'DEVELOP', 'DINNER', 'DOG', 'DOWNSIZE', 'DRAG',
       'EAT', 'EDIT', 'ELEVATOR', 'FINE', 'FLOAT', 'FOREIGNER', 'GUESS',
       'HALLOWEEN', 'HOSPITAL', 'HURDLE/TRIP', 'JEWELRY', 'KNIGHT',
       'LOCK', 'LUNCH', 'MAPLE', 'MEAT', 'MECHANIC', 'MICROSCOPE',
       'MOVIE', 'NOON', 'PARTY', 'PATIENT', 'RAZOR', 'RIVER',
       'ROCKINGCHAIR', 'SHAVE', 'SINK', 'SQUEEZE', 'THEY', 'TWINS',
       'WHATFOR'], dtype=object)

In [47]:
print(traindf['Gloss'].nunique())
print(valdf['Gloss'].nunique())
print(testdf['Gloss'].nunique())

50
50
50


In [48]:
cap = cv2.VideoCapture(traindf.Path[0])
#cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=2) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        # rostro
        mp_drawing.draw_landmarks(
            frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(0, 255, 255),
                                   thickness=1,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2))

        # Mano izquieda (azul)
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Mano derecha (verde)
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Postura
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))
        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [49]:
# Define a function to extract keypoints from a single video file
def extract_keypoints_from_file(filepath):
    # Initialize a MediaPipe Holistic model
    holistic = mp.solutions.holistic.Holistic(static_image_mode=False,
                                              model_complexity=2,
                                              min_detection_confidence=0.5,
                                              min_tracking_confidence=0.5)

    # Open the video file
    cap = cv2.VideoCapture(filepath)

    # Initialize an empty list to store the keypoints for each frame
    keypoints = []

    # Loop through the frames of the video
    while cap.isOpened():
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB color space
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Use the Holistic model to detect landmarks for the face, pose, and hands
        results = holistic.process(frame)

        # Extract the keypoints from the results object and append them to the keypoints list
        keypoints.append(extract_keypoints(results))

    # Release the video capture object and the Holistic model
    cap.release()
    holistic.close()

    return np.array(keypoints)

In [50]:
extract_keypoints_from_file(traindf.Path[0]).shape

(100, 1662)

In [51]:
cap = cv2.VideoCapture(traindf.Path[0])
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print( length )

100


In [52]:
type(extract_keypoints_from_file(traindf.Path[643]))

numpy.ndarray

In [53]:
traindf.count()

Participant ID    950
Video file        950
Gloss             950
ASL-LEX Code      950
Path              950
Frequency         950
dtype: int64

In [54]:
def frames_from_file(filepath):
    cap = cv2.VideoCapture(filepath)
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    length = round(frames / fps)
    return frames, fps, length

In [55]:
frames_from_file(traindf.Path[0])

(100, 31, 3)

In [58]:
tqdm.pandas()
traindf[['Frames', 'FPS', 'Length']] = traindf['Path'].progress_apply(lambda x: pd.Series(frames_from_file(x)))

100%|████████████████████████████████████████| 950/950 [00:05<00:00, 179.37it/s]


In [62]:
traindf['Frames'].max()

540

In [60]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
0,P52,07157565148825373-seedAXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,100,31,3
1,P28,7179300005186042-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,81,30,3
2,P29,16216064841959765-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,47,30,2
3,P37,6193814382865199-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,67,30,2
4,P40,5947453960317015-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,89,30,3


In [61]:
traindf.describe()

,Frequency,Frames,FPS,Length
count,950.000000,950.000000,950.000000,950.000000
mean,19.082105,87.755789,29.354737,3.016842
std,1.316026,46.654745,2.197195,1.568167
min,18.000000,25.000000,11.000000,1.000000
25%,18.000000,60.000000,29.000000,2.000000
50%,19.000000,77.000000,30.000000,3.000000
75%,20.000000,97.000000,30.000000,3.000000
max,24.000000,540.000000,31.000000,18.000000


In [63]:
traindf = traindf[(traindf['Length'] > 1) & (traindf['FPS'] > 28) &(traindf['Frames'] < 131) ]
#filtered_df = traindf[(traindf['Frames'] > 0) & (traindf['Frames'] < 121)]
#filtered_df = traindf[traindf['Frames'] > 0]
traindf= traindf.reset_index(drop=True)
traindf.describe()

,Frequency,Frames,FPS,Length
count,804.000000,804.000000,804.000000,804.000000
mean,19.084577,76.113184,29.680348,2.593284
std,1.305343,20.091965,0.479790,0.671427
min,18.000000,45.000000,29.000000,2.000000
25%,18.000000,60.000000,29.000000,2.000000
50%,19.000000,74.000000,30.000000,2.000000
75%,20.000000,88.000000,30.000000,3.000000
max,24.000000,130.000000,31.000000,4.000000


In [64]:
traindf['Frequency'] = traindf['Gloss'].map(traindf['Gloss'].value_counts())
#traindf = traindf.sort_values(by='Frequency',)

traindf.describe()

,Frequency,Frames,FPS,Length
count,804.000000,804.000000,804.000000,804.000000
mean,16.261194,76.113184,29.680348,2.593284
std,1.722009,20.091965,0.479790,0.671427
min,13.000000,45.000000,29.000000,2.000000
25%,15.000000,60.000000,29.000000,2.000000
50%,16.000000,74.000000,30.000000,2.000000
75%,17.000000,88.000000,30.000000,3.000000
max,20.000000,130.000000,31.000000,4.000000


In [93]:
traindf.head(10)

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
0,P52,07157565148825373-seedAXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,100,31,3
1,P28,7179300005186042-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,81,30,3
2,P29,16216064841959765-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,47,30,2
3,P37,6193814382865199-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,67,30,2
4,P40,5947453960317015-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,89,30,3
5,P40,929570016067665-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,112,29,4
6,P16,904767261814883-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,64,30,2
7,P43,016134052760367945-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,75,30,2
8,P40,23099527328070546-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,91,29,3
9,P37,21310388087265242-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,69,30,2


In [65]:
print(traindf['Gloss'].nunique())

50


In [66]:
tqdm.pandas()
valdf[['Frames', 'FPS', 'Length']] = valdf['Path'].progress_apply(lambda x: pd.Series(frames_from_file(x)))

100%|█████████████████████████████████████████| 191/191 [00:12<00:00, 15.47it/s]


In [67]:
valdf['Frames'].max()

207

In [68]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
0,P39,19778675091674147-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,121,30,4
1,P26,8581142177964065-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,81,29,3
2,P39,3877478645046861-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,120,30,4
3,P21,8521417940364975-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,71,29,2
4,P5,9990244640190733-BACKPACK.mp4,BACKPACK,G_03_091,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3,93,29,3


In [69]:
valdf.describe()

,Frequency,Frames,FPS,Length
count,191.000000,191.000000,191.000000,191.000000
mean,3.963351,88.905759,29.706806,3.000000
std,0.770316,36.214006,0.456423,1.248157
min,3.000000,37.000000,29.000000,1.000000
25%,3.000000,61.500000,29.000000,2.000000
50%,4.000000,76.000000,30.000000,3.000000
75%,4.000000,117.500000,30.000000,4.000000
max,6.000000,207.000000,30.000000,7.000000


In [70]:
valdf = valdf[(valdf['Length'] > 1) & (valdf['FPS'] > 28) &(valdf['Frames'] < 131) ]
#filtered_df = traindf[(traindf['Frames'] > 0) & (traindf['Frames'] < 121)]
#filtered_df = traindf[traindf['Frames'] > 0]
valdf = valdf.reset_index(drop=True)
valdf.describe()

,Frequency,Frames,FPS,Length
count,162.000000,162.000000,162.000000,162.000000
mean,3.993827,78.203704,29.691358,2.623457
std,0.760001,23.787397,0.463365,0.772311
min,3.000000,45.000000,29.000000,2.000000
25%,3.000000,59.250000,29.000000,2.000000
50%,4.000000,71.500000,30.000000,2.000000
75%,4.000000,92.000000,30.000000,3.000000
max,6.000000,130.000000,30.000000,4.000000


In [71]:
valdf['Frequency'] = valdf['Gloss'].map(valdf['Gloss'].value_counts())
#valdf = valdf.sort_values(by='Frequency',)

valdf.describe()

,Frequency,Frames,FPS,Length
count,162.000000,162.000000,162.000000,162.000000
mean,3.506173,78.203704,29.691358,2.623457
std,0.907162,23.787397,0.463365,0.772311
min,2.000000,45.000000,29.000000,2.000000
25%,3.000000,59.250000,29.000000,2.000000
50%,4.000000,71.500000,30.000000,2.000000
75%,4.000000,92.000000,30.000000,3.000000
max,5.000000,130.000000,30.000000,4.000000


In [72]:
print(valdf['Gloss'].nunique())

50


In [74]:
tqdm.pandas()
testdf[['Frames', 'FPS', 'Length']] = testdf['Path'].progress_apply(lambda x: pd.Series(frames_from_file(x)))

100%|█████████████████████████████████████████| 729/729 [00:34<00:00, 20.96it/s]


In [75]:
testdf['Frames'].max()

234

In [76]:
testdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
0,P15,33721516025652254-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15,196,29,7
1,P47,519067006979435-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15,102,26,4
2,P35,5104381603195376-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15,70,29,2
3,P42,44458614013793873-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15,57,30,2
4,P42,9604797909481075-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15,66,30,2


In [77]:
testdf.describe()

,Frequency,Frames,FPS,Length
count,729.000000,729.000000,729.000000,729.000000
mean,14.978052,75.192044,28.696845,2.624143
std,2.229192,27.848864,3.113689,1.003439
min,9.000000,10.000000,14.000000,0.000000
25%,14.000000,56.000000,29.000000,2.000000
50%,15.000000,73.000000,30.000000,3.000000
75%,16.000000,90.000000,30.000000,3.000000
max,20.000000,234.000000,31.000000,8.000000


In [78]:
testdf = testdf[(testdf['Length'] > 1) & (testdf['FPS'] > 28) &(testdf['Frames'] < 131) ]
#filtered_df = traindf[(traindf['Frames'] > 0) & (traindf['Frames'] < 121)]
#filtered_df = traindf[traindf['Frames'] > 0]
testdf = testdf.reset_index(drop=True)
testdf.describe()

,Frequency,Frames,FPS,Length
count,534.000000,534.000000,534.000000,534.000000
mean,15.014981,77.256554,29.702247,2.619850
std,2.217904,19.420007,0.526337,0.670792
min,9.000000,45.000000,29.000000,2.000000
25%,14.000000,61.000000,29.000000,2.000000
50%,15.000000,75.000000,30.000000,3.000000
75%,16.000000,91.000000,30.000000,3.000000
max,20.000000,130.000000,31.000000,4.000000


In [79]:
testdf['Frequency'] = testdf['Gloss'].map(testdf['Gloss'].value_counts())
#testdf = testdf.sort_values(by='Frequency',)

testdf.describe()

,Frequency,Frames,FPS,Length
count,534.000000,534.000000,534.000000,534.000000
mean,11.254682,77.256554,29.702247,2.619850
std,2.311308,19.420007,0.526337,0.670792
min,5.000000,45.000000,29.000000,2.000000
25%,10.000000,61.000000,29.000000,2.000000
50%,12.000000,75.000000,30.000000,3.000000
75%,13.000000,91.000000,30.000000,3.000000
max,15.000000,130.000000,31.000000,4.000000


In [80]:
print(testdf['Gloss'].nunique())

50


In [81]:
keypoints_series = traindf['Path'].progress_apply(extract_keypoints_from_file)

100%|███████████████████████████████████████| 804/804 [2:27:14<00:00, 10.99s/it]


In [110]:
keypoints_series.shape

(804,)

In [84]:
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/train_keypoints.npy", keypoints_series)

In [108]:
import numpy as np
keypoints_series=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/train_keypoints.npy",allow_pickle=True)

In [111]:
max(keypoints.shape[0] for keypoints in keypoints_series)

130

In [112]:
min(keypoints.shape[0] for keypoints in keypoints_series)


45

In [113]:
max_len = traindf['Frames'].max()

In [114]:
max_len

130

In [94]:
#keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))
#for i, keypoints in enumerate(keypoints_series):
#    keypoints_np[i, :keypoints.shape[0], :] = keypoints

In [115]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(keypoints_series):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]

In [116]:
keypoints_np.shape

(804, 130, 1662)

In [117]:
print(keypoints_np[6])

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.49487716  0.39238185 -1.29218626 ...  0.          0.
   0.        ]
 [ 0.49096876  0.39365393 -1.30090356 ...  0.          0.
   0.        ]
 [ 0.48817223  0.39431143 -1.3236835  ...  0.          0.
   0.        ]]


In [119]:
keypoints_np.shape

(804, 130, 1662)

In [120]:
keypoints_series = valdf['Path'].progress_apply(extract_keypoints_from_file)

100%|█████████████████████████████████████████| 162/162 [29:55<00:00, 11.08s/it]


In [121]:
keypoints_series.shape

(162,)

In [122]:
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/val_keypoints.npy", keypoints_series)

In [123]:
import numpy as np
keypoints_series=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/val_keypoints.npy",allow_pickle=True)

In [124]:
keypoints_series.shape

(162,)

In [125]:
#keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))
#for i, keypoints in enumerate(keypoints_series):
#    keypoints_np[i, :keypoints.shape[0], :] = keypoints

In [126]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(keypoints_series):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]

In [127]:
keypoints_np.shape

(162, 130, 1662)

In [128]:
keypoints_np.shape

(162, 130, 1662)

In [129]:
keypoints_np

array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.5595299 ,  0.3205725 , -1.27106977, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.55954963,  0.32069033, -1.25959992, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.55960971,  0.3209095 , -1.23169589, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.5183838 ,  0.33223236, -1.15552998, ...,  

In [112]:
keypoints_series = testdf['Path'].progress_apply(extract_keypoints_from_file)

100%|█████████████████████████████████| 705/705 [2:00:53<00:00, 10.29s/it]


In [113]:
keypoints_series.shape

(705,)

In [116]:
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/test_keypoints.npy", keypoints_np)

In [ ]:
import numpy as np
keypoints_series=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/test_keypoints.npy", allow_pickle=True)

In [ ]:
keypoints_series.shape

In [ ]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(keypoints_series):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]

In [114]:
keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))
for i, keypoints in enumerate(keypoints_series):
    keypoints_np[i, :keypoints.shape[0], :] = keypoints

In [115]:
keypoints_np.shape

(705, 130, 1662)

In [118]:
keypoints_np.shape

(705, 130, 1662)

# Preprocessing

In [19]:
import numpy as np
train_keypoints=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/train_keypoints.npy",allow_pickle=True)
val_keypoints =np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/val_keypoints.npy",allow_pickle=True)
#test_keypoints=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/test_keypoints.npy")

print('Training Keypoints Shape:', train_keypoints.shape)
print('Validation Kepoints Shape:', val_keypoints.shape)
#print('Test Keypoints Shape:',test_keypoints.shape)

Training Keypoints Shape: (804,)
Validation Kepoints Shape: (162,)


In [20]:
max_len=130

In [21]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(train_keypoints), max_len, 1662))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(train_keypoints):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]
    
train_keypoints = keypoints_np
print('Training Keypoints Shape:', train_keypoints.shape)

Training Keypoints Shape: (804, 130, 1662)


In [22]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(val_keypoints), max_len, 1662))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(val_keypoints):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]
    
val_keypoints = keypoints_np
print('Validation Keypoints Shape:', val_keypoints.shape)

Validation Keypoints Shape: (162, 130, 1662)


In [5]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(test_keypoints), max_len, 1662))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(test_keypoints):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]
    
test_keypoints = keypoints_np
print('Test Keypoints Shape:', test_keypoints.shape)

NameError: name 'test_keypoints' is not defined

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from keras.utils import to_categorical
import numpy as np
from keras.preprocessing.sequence import pad_sequences

In [6]:
traindf.head()

NameError: name 'traindf' is not defined

In [138]:
encoder = OrdinalEncoder()

In [139]:
traindf['Cat_label'] = encoder.fit_transform(traindf[['Gloss']]).astype(int)
traindf = traindf.sort_values(by='Gloss',)

In [140]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
0,P52,07157565148825373-seedAXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,100,31,3,0
17,P37,29360158406046777-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,73,30,2,0
16,P27,053394218351220823-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,52,29,2,0
15,P40,10598328043940142-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,81,29,3,0
14,P50,3937261764308986-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,72,30,2,0


In [141]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
789,P40,6445331634562388-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16,89,30,3,49
788,P37,6237575353180616-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16,56,30,2,49
802,P40,5268072837528903-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16,92,29,3,49
794,P37,16185522171162914-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16,64,30,2,49
803,P40,6363286086951516-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16,65,29,2,49


In [142]:
valdf['Cat_label'] = encoder.fit_transform(valdf[['Gloss']]).astype(int)
valdf = valdf.sort_values(by='Gloss',)

In [143]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
0,P39,19778675091674147-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,121,30,4,0
1,P26,8581142177964065-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,81,29,3,0
2,P39,3877478645046861-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,120,30,4,0
3,P21,8521417940364975-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,71,29,2,0
4,P5,9990244640190733-BACKPACK.mp4,BACKPACK,G_03_091,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3,93,29,3,1


In [144]:
valdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
159,P39,0283886564670357-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5,121,30,4,49
160,P21,6142521746642153-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5,57,30,2,49
157,P26,032677896012150764-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5,81,29,3,49
158,P5,6523145816470133-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5,116,30,4,49
161,P12,06408604416165864-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5,71,30,2,49


In [145]:
testdf['Cat_label'] = encoder.fit_transform(testdf[['Gloss']]).astype(int)
testdf = testdf.sort_values(by='Gloss',)

In [146]:
testdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
0,P35,5104381603195376-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,11,70,29,2,0
1,P42,44458614013793873-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,11,57,30,2,0
2,P42,9604797909481075-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,11,66,30,2,0
3,P9,30728048195204827-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,11,96,30,3,0
4,P6,28522130623160047-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,11,54,29,2,0


In [147]:
testdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
522,P6,6734748834983595-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13,66,29,2,49
521,P17,33051835760678294-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13,52,31,2,49
532,P42,2748457214117681-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13,74,30,2,49
526,P42,9684934734400592-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13,89,30,3,49
533,P15,540014801916062-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13,79,29,3,49


In [148]:
label_map = {label:num for num, label in enumerate(traindf['Gloss'].unique())}

In [149]:
label_map

{'AXE': 0,
 'BACKPACK': 1,
 'BASKETBALL': 2,
 'BEE': 3,
 'BELT': 4,
 'BITE': 5,
 'BREAKFAST': 6,
 'CANCER': 7,
 'CHRISTMAS': 8,
 'CONFUSED': 9,
 'DARK': 10,
 'DEAF': 11,
 'DECIDE': 12,
 'DEMAND': 13,
 'DEVELOP': 14,
 'DINNER': 15,
 'DOG': 16,
 'DOWNSIZE': 17,
 'DRAG': 18,
 'EAT': 19,
 'EDIT': 20,
 'ELEVATOR': 21,
 'FINE': 22,
 'FLOAT': 23,
 'FOREIGNER': 24,
 'GUESS': 25,
 'HALLOWEEN': 26,
 'HOSPITAL': 27,
 'HURDLE/TRIP': 28,
 'JEWELRY': 29,
 'KNIGHT': 30,
 'LOCK': 31,
 'LUNCH': 32,
 'MAPLE': 33,
 'MEAT': 34,
 'MECHANIC': 35,
 'MICROSCOPE': 36,
 'MOVIE': 37,
 'NOON': 38,
 'PARTY': 39,
 'PATIENT': 40,
 'RAZOR': 41,
 'RIVER': 42,
 'ROCKINGCHAIR': 43,
 'SHAVE': 44,
 'SINK': 45,
 'SQUEEZE': 46,
 'THEY': 47,
 'TWINS': 48,
 'WHATFOR': 49}

In [150]:
traindf.sample(10)

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
685,P11,8561671103002648-RIVER.mp4,RIVER,A_03_035,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15,68,29,2,42
454,P27,9543121609158431-HOSPITAL.mp4,HOSPITAL,B_02_026,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,17,63,29,2,27
618,P37,1012683026825647-NOON.mp4,NOON,G_02_014,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,17,62,30,2,38
673,P36,34327843271669756-RAZOR 2.mp4,RAZOR,K_03_103,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13,75,30,2,41
476,P50,22335744767716914-HURDLE-TRIP.mp4,HURDLE/TRIP,G_01_036,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,17,87,30,3,28
482,P23,7185958868464457-JEWELRY.mp4,JEWELRY,H_03_040,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15,95,30,3,29
321,P31,6460191084812579-EAT.mp4,EAT,B_02_002,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,14,79,30,3,19
307,P37,20831865620547463-DRAG.mp4,DRAG,J_02_039,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16,60,29,2,18
735,P37,43245204383901537-SINK.mp4,SINK,E_02_042,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,14,71,30,2,45
118,P40,23598266126803202-BREAKFAST.mp4,BREAKFAST,J_01_053,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,85,29,3,6


In [6]:
Ytrain = encoder.fit_transform(traindf[['Gloss']])
Ytest = encoder.fit_transform(testdf[['Gloss']])
Yval = encoder.fit_transform(valdf[['Gloss']])

NameError: name 'encoder' is not defined

In [152]:
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/train_labels_array_1D.npy", Ytrain)
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/test_labels_array_1D.npy", Ytest)
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/val_labels_array_1D.npy", Yval)

In [24]:
Ytrain=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/train_labels_array_1D.npy")
Ytest=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/test_labels_array_1D.npy")
Yval=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/val_labels_array_1D.npy")


In [25]:
Xtrain = train_keypoints
#Xtest = test_keypoints
Xval = val_keypoints

In [26]:
n_classes = 50
print("Shape before one-hot encoding: ", Ytrain.shape)
Ytrain = to_categorical(Ytrain, n_classes)
print("Shape after one-hot encoding: ", Ytrain.shape)
print("Shape before one-hot encoding: ", Ytest.shape)
Ytest = to_categorical(Ytest, n_classes)
print("Shape after one-hot encoding: ", Ytest.shape)
print("Shape before one-hot encoding: ", Yval.shape)
Yval = to_categorical(Yval, n_classes)
print("Shape after one-hot encoding: ", Yval.shape)

Shape before one-hot encoding:  (804, 1)
Shape after one-hot encoding:  (804, 50)
Shape before one-hot encoding:  (534, 1)
Shape after one-hot encoding:  (534, 50)
Shape before one-hot encoding:  (162, 1)
Shape after one-hot encoding:  (162, 50)


In [27]:
type(Ytrain)

numpy.ndarray

In [28]:
Ytrain.shape

(804, 50)

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.2,
                                                    random_state=10)

In [29]:
from sklearn.preprocessing import MinMaxScaler


num_samples, num_frames, num_keypoints = Xtrain.shape
Xtrain = Xtrain.reshape(num_samples, -1)

# Create a MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
Xtrain = scaler.fit_transform(Xtrain)

# Reshape the scaled data back to the original shape
Xtrain = Xtrain.reshape(num_samples, num_frames, num_keypoints)

In [30]:
Xtrain.shape

(804, 130, 1662)

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(input_data_scaled,
                                                    Y,
                                                    test_size=0.,
                                                    random_state=10)

NameError: name 'input_data_scaled' is not defined

In [31]:
Xtrain.shape

(804, 130, 1662)

In [32]:
Xtrain

array([[[0.        , 0.        , 1.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 1.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 1.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.32836917, 0.43633427, 0.59659172, ..., 0.        ,
         0.        , 0.94466859],
        [0.32857075, 0.4358778 , 0.6248101 , ..., 0.        ,
         0.        , 1.        ],
        [0.32915611, 0.43495438, 0.63639134, ..., 0.        ,
         0.        , 0.97963139]],

       [[0.        , 0.        , 1.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 1.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 1.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.42377348, 0.31397443, 0.37976738, ..., 0.        ,
         0.        , 0.94466859],
        [0.4

# Model

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam ,Adagrad, Adadelta, SGD, Lion
import os


In [34]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [35]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

In [36]:
max_len = 130

In [18]:
model = Sequential(name='Sequential')
act_function = 'tanh'
model.add(LSTM(180, return_sequences=True, activation=act_function, input_shape=(130,1662)))
model.add(LSTM(90, return_sequences=True, activation=act_function))
model.add(LSTM(180, return_sequences=True, activation=act_function))
model.add(LSTM(90, return_sequences=False,activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(45, activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(50, activation=act_function))
model.summary()

optimizer = SGD(learning_rate=.00001,nesterov=True)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 64)

2023-08-07 21:31:15.975071: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1037 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1


Model: "Sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 130, 180)          1326960   
                                                                 
 lstm_1 (LSTM)               (None, 130, 90)           97560     
                                                                 
 lstm_2 (LSTM)               (None, 130, 180)          195120    
                                                                 
 lstm_3 (LSTM)               (None, 90)                97560     
                                                                 
 dense (Dense)               (None, 90)                8190      
                                                                 
 dense_1 (Dense)             (None, 90)                8190      
                                                                 
 dense_2 (Dense)             (None, 90)                8

2023-08-07 21:31:23.890758: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-07 21:31:24.039649: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xa6a3900 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-07 21:31:24.039679: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-08-07 21:31:24.100011: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


11/11 [==============================] - 10s 360ms/step - loss: 8.4307 - categorical_accuracy: 0.0171 - val_loss: 5.4406 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
11/11 [==============================] - 2s 218ms/step - loss: 7.6683 - categorical_accuracy: 0.0311 - val_loss: 5.3028 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
11/11 [==============================] - 2s 220ms/step - loss: 7.6718 - categorical_accuracy: 0.0249 - val_loss: 5.0410 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
11/11 [==============================] - 2s 221ms/step - loss: 8.3987 - categorical_accuracy: 0.0187 - val_loss: 5.0319 - val_categorical_accuracy: 0.0000e+00
Epoch 5/1000
11/11 [==============================] - 2s 221ms/step - loss: 7.4197 - categorical_accuracy: 0.0280 - val_loss: 5.5538 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
11/11 [==============================] - 2s 219ms/step - loss: 8.0407 - categorical_accuracy: 0.0311 - val_loss: 7.9545 - val_categorical_a

KeyboardInterrupt: 

In [18]:
model = Sequential(name='Sequential')
act_function = 'tanh'
model.add(GRU(64, return_sequences=True, activation=act_function, input_shape=(130,1662)))
model.add(GRU(64, return_sequences=False, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dense(50, activation=act_function))
optimizer = Adagrad(learning_rate=.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 64)
model.summary()

2023-08-07 21:49:05.468776: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1037 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1


Epoch 1/1000


2023-08-07 21:49:09.595786: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-07 21:49:09.643909: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8864929fe0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-07 21:49:09.643936: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-08-07 21:49:09.647840: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-07 21:49:09.756578: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


11/11 [==============================] - 5s 178ms/step - loss: 7.8501 - categorical_accuracy: 0.0311 - val_loss: 8.1508 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
11/11 [==============================] - 1s 82ms/step - loss: 8.1865 - categorical_accuracy: 0.0218 - val_loss: 9.0874 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
11/11 [==============================] - 1s 82ms/step - loss: 7.9489 - categorical_accuracy: 0.0171 - val_loss: 9.2725 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
11/11 [==============================] - 1s 82ms/step - loss: 8.4394 - categorical_accuracy: 0.0264 - val_loss: 7.0158 - val_categorical_accuracy: 0.0435
Epoch 5/1000
11/11 [==============================] - 1s 82ms/step - loss: 7.9150 - categorical_accuracy: 0.0264 - val_loss: 6.5328 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
11/11 [==============================] - 1s 83ms/step - loss: 7.8367 - categorical_accuracy: 0.0249 - val_loss: 7.1868 - val_categorical_accuracy: 0

KeyboardInterrupt: 

In [57]:
model = Sequential()

act_function = 'LeakyReLU'
model.add(LSTM(64, return_sequences=True, activation=act_function, input_shape=(130,1662)))
model.add(GRU(128, return_sequences=True, activation=act_function))
model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dense(32, activation=act_function))
model.add(Dense(50, activation='softmax'))
optimizer = SGD(learning_rate=.0000005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 32, callbacks=[tb_callback])
model.summary()

Epoch 1/1000
21/21 [==============================] - 16s 566ms/step - loss: 3.9102 - categorical_accuracy: 0.0218 - val_loss: 3.9294 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
21/21 [==============================] - 11s 541ms/step - loss: 3.9102 - categorical_accuracy: 0.0218 - val_loss: 3.9294 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
 7/21 [=========>....................] - ETA: 7s - loss: 3.9032 - categorical_accuracy: 0.0223

KeyboardInterrupt: 

In [41]:
model = Sequential() #good, overfitting

act_function = 'LeakyReLU'
model.add(GRU(128, return_sequences=False, activation=act_function, input_shape=(130,1662)))
#model.add(GRU(128, return_sequences=True, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(128, activation=act_function, input_shape=(130,1662)))
model.add(Dense(128, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.00025)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 32, callbacks=[tb_callback])
model.summary()

Epoch 1/1000


2023-08-07 23:26:13.531247: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


21/21 [==============================] - 7s 260ms/step - loss: 3.9072 - categorical_accuracy: 0.0249 - val_loss: 4.4004 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
21/21 [==============================] - 4s 200ms/step - loss: 3.8445 - categorical_accuracy: 0.0280 - val_loss: 5.2406 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
21/21 [==============================] - 4s 210ms/step - loss: 3.7973 - categorical_accuracy: 0.0311 - val_loss: 5.7973 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
21/21 [==============================] - 4s 205ms/step - loss: 3.7802 - categorical_accuracy: 0.0280 - val_loss: 6.3050 - val_categorical_accuracy: 0.0000e+00
Epoch 5/1000
21/21 [==============================] - 4s 211ms/step - loss: 3.7318 - categorical_accuracy: 0.0311 - val_loss: 6.7130 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
21/21 [==============================] - 4s 212ms/step - loss: 3.7243 - categorical_accuracy: 0.0358 - val_loss: 6.5412 - val_categorical_ac

21/21 [==============================] - 5s 231ms/step - loss: 1.9802 - categorical_accuracy: 0.4277 - val_loss: 35.2716 - val_categorical_accuracy: 0.0000e+00
Epoch 49/1000
21/21 [==============================] - 5s 232ms/step - loss: 1.9189 - categorical_accuracy: 0.4526 - val_loss: 35.2482 - val_categorical_accuracy: 0.0000e+00
Epoch 50/1000
21/21 [==============================] - 5s 230ms/step - loss: 1.8469 - categorical_accuracy: 0.4650 - val_loss: 37.3292 - val_categorical_accuracy: 0.0000e+00
Epoch 51/1000
21/21 [==============================] - 5s 235ms/step - loss: 1.8254 - categorical_accuracy: 0.4541 - val_loss: 39.3376 - val_categorical_accuracy: 0.0000e+00
Epoch 52/1000
21/21 [==============================] - 5s 228ms/step - loss: 1.7407 - categorical_accuracy: 0.4914 - val_loss: 38.7546 - val_categorical_accuracy: 0.0000e+00
Epoch 53/1000
21/21 [==============================] - 5s 232ms/step - loss: 1.6727 - categorical_accuracy: 0.5163 - val_loss: 41.2606 - val_cat

21/21 [==============================] - 5s 236ms/step - loss: 0.5081 - categorical_accuracy: 0.8523 - val_loss: 94.4309 - val_categorical_accuracy: 0.0000e+00
Epoch 96/1000
21/21 [==============================] - 5s 237ms/step - loss: 0.4837 - categorical_accuracy: 0.8351 - val_loss: 95.2666 - val_categorical_accuracy: 0.0000e+00
Epoch 97/1000
21/21 [==============================] - 5s 234ms/step - loss: 0.4560 - categorical_accuracy: 0.8538 - val_loss: 95.4708 - val_categorical_accuracy: 0.0000e+00
Epoch 98/1000
21/21 [==============================] - 5s 235ms/step - loss: 0.4226 - categorical_accuracy: 0.8678 - val_loss: 97.3364 - val_categorical_accuracy: 0.0000e+00
Epoch 99/1000
21/21 [==============================] - 5s 231ms/step - loss: 0.4609 - categorical_accuracy: 0.8554 - val_loss: 99.6497 - val_categorical_accuracy: 0.0000e+00
Epoch 100/1000
21/21 [==============================] - 5s 233ms/step - loss: 0.4896 - categorical_accuracy: 0.8585 - val_loss: 102.9560 - val_c

KeyboardInterrupt: 

In [37]:
model = Sequential()

act_function = 'LeakyReLU'
model.add(GRU(128, return_sequences=False, activation=act_function, input_shape=(130,1662)))
#model.add(GRU(128, return_sequences=True, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(128, activation=act_function, input_shape=(130,1662)))
model.add(Dense(128, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dropout(.2))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.00025)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 32, callbacks=[tb_callback])
model.summary()

2023-08-07 23:51:45.622929: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1739 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1


Epoch 1/1000


2023-08-07 23:51:48.945654: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f1a740b3f40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-07 23:51:48.945700: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-08-07 23:51:48.952538: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-07 23:51:48.972420: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-07 23:51:49.040054: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


21/21 [==============================] - 8s 252ms/step - loss: 3.9271 - categorical_accuracy: 0.0295 - val_loss: 4.2207 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
21/21 [==============================] - 4s 209ms/step - loss: 3.9170 - categorical_accuracy: 0.0218 - val_loss: 4.3225 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
21/21 [==============================] - 4s 209ms/step - loss: 3.8791 - categorical_accuracy: 0.0171 - val_loss: 4.6208 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
21/21 [==============================] - 4s 214ms/step - loss: 3.8522 - categorical_accuracy: 0.0280 - val_loss: 4.7824 - val_categorical_accuracy: 0.0000e+00
Epoch 5/1000
21/21 [==============================] - 4s 213ms/step - loss: 3.8406 - categorical_accuracy: 0.0358 - val_loss: 4.7065 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
21/21 [==============================] - 5s 218ms/step - loss: 3.8190 - categorical_accuracy: 0.0295 - val_loss: 4.9908 - val_categorical_ac

21/21 [==============================] - 5s 245ms/step - loss: 2.1086 - categorical_accuracy: 0.3297 - val_loss: 11.2346 - val_categorical_accuracy: 0.0000e+00
Epoch 49/1000
21/21 [==============================] - 5s 241ms/step - loss: 2.0561 - categorical_accuracy: 0.3608 - val_loss: 10.6040 - val_categorical_accuracy: 0.0000e+00
Epoch 50/1000
21/21 [==============================] - 5s 233ms/step - loss: 2.0562 - categorical_accuracy: 0.3530 - val_loss: 11.5774 - val_categorical_accuracy: 0.0000e+00
Epoch 51/1000
21/21 [==============================] - 5s 234ms/step - loss: 1.9937 - categorical_accuracy: 0.3841 - val_loss: 11.7444 - val_categorical_accuracy: 0.0000e+00
Epoch 52/1000
21/21 [==============================] - 5s 229ms/step - loss: 1.9682 - categorical_accuracy: 0.3826 - val_loss: 11.9445 - val_categorical_accuracy: 0.0000e+00
Epoch 53/1000
21/21 [==============================] - 5s 235ms/step - loss: 1.9129 - categorical_accuracy: 0.4090 - val_loss: 12.0834 - val_cat

21/21 [==============================] - 5s 248ms/step - loss: 0.8217 - categorical_accuracy: 0.7170 - val_loss: 20.6062 - val_categorical_accuracy: 0.0062
Epoch 96/1000
21/21 [==============================] - 5s 255ms/step - loss: 0.8497 - categorical_accuracy: 0.7216 - val_loss: 21.2923 - val_categorical_accuracy: 0.0000e+00
Epoch 97/1000
21/21 [==============================] - 5s 249ms/step - loss: 0.8300 - categorical_accuracy: 0.7107 - val_loss: 19.6096 - val_categorical_accuracy: 0.0000e+00
Epoch 98/1000
21/21 [==============================] - 6s 267ms/step - loss: 0.7510 - categorical_accuracy: 0.7543 - val_loss: 19.1022 - val_categorical_accuracy: 0.0000e+00
Epoch 99/1000
21/21 [==============================] - 5s 248ms/step - loss: 0.7992 - categorical_accuracy: 0.7154 - val_loss: 20.5280 - val_categorical_accuracy: 0.0000e+00
Epoch 100/1000
21/21 [==============================] - 5s 240ms/step - loss: 0.7177 - categorical_accuracy: 0.7574 - val_loss: 20.5536 - val_catego

21/21 [==============================] - 5s 237ms/step - loss: 0.5578 - categorical_accuracy: 0.8429 - val_loss: 26.3455 - val_categorical_accuracy: 0.0000e+00
Epoch 143/1000
21/21 [==============================] - 5s 235ms/step - loss: 0.5668 - categorical_accuracy: 0.8196 - val_loss: 24.4923 - val_categorical_accuracy: 0.0000e+00
Epoch 144/1000
21/21 [==============================] - 5s 233ms/step - loss: 0.4871 - categorical_accuracy: 0.8507 - val_loss: 26.0941 - val_categorical_accuracy: 0.0000e+00
Epoch 145/1000
21/21 [==============================] - 5s 232ms/step - loss: 0.4818 - categorical_accuracy: 0.8398 - val_loss: 24.5604 - val_categorical_accuracy: 0.0000e+00
Epoch 146/1000
21/21 [==============================] - 5s 245ms/step - loss: 0.4451 - categorical_accuracy: 0.8383 - val_loss: 24.6997 - val_categorical_accuracy: 0.0000e+00
Epoch 147/1000
 8/21 [==========>...................] - ETA: 3s - loss: 0.3756 - categorical_accuracy: 0.8906

KeyboardInterrupt: 

In [19]:
model = Sequential()

act_function = 'elu'
model.add(GRU(32, return_sequences=False, activation=act_function, dropout=.5,input_shape=(130,1662)))
#model.add(GRU(128, return_sequences=True, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(32, activation=act_function))
#model.add(Dense(64, activation=act_function))
model.add(Dense(16, activation=act_function))
model.add(Dropout(.3))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.00005,global_clipnorm=1)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 128, callbacks=[tb_callback])
model.summary()

2023-08-08 00:42:54.535650: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1739 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1


Epoch 1/1000


2023-08-08 00:42:58.646820: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-08 00:42:58.750163: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f53c892a510 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-08 00:42:58.750197: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-08-08 00:42:58.755965: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-08 00:42:58.830948: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6/6 [==============================] - 5s 422ms/step - loss: 3.9337 - categorical_accuracy: 0.0233 - val_loss: 4.0035 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
6/6 [==============================] - 1s 126ms/step - loss: 3.9116 - categorical_accuracy: 0.0280 - val_loss: 3.9327 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
6/6 [==============================] - 1s 129ms/step - loss: 3.9111 - categorical_accuracy: 0.0295 - val_loss: 4.0119 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
6/6 [==============================] - 1s 126ms/step - loss: 3.9109 - categorical_accuracy: 0.0249 - val_loss: 4.0355 - val_categorical_accuracy: 0.0000e+00
Epoch 5/1000
6/6 [==============================] - 1s 127ms/step - loss: 3.9106 - categorical_accuracy: 0.0280 - val_loss: 4.0624 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
6/6 [==============================] - 1s 128ms/step - loss: 3.9100 - categorical_accuracy: 0.0249 - val_loss: 4.0626 - val_categorical_accuracy: 0.00

6/6 [==============================] - 1s 130ms/step - loss: 3.8928 - categorical_accuracy: 0.0295 - val_loss: 4.1070 - val_categorical_accuracy: 0.0000e+00
Epoch 50/1000
6/6 [==============================] - 1s 129ms/step - loss: 3.8933 - categorical_accuracy: 0.0187 - val_loss: 4.1108 - val_categorical_accuracy: 0.0000e+00
Epoch 51/1000
6/6 [==============================] - 1s 132ms/step - loss: 3.8900 - categorical_accuracy: 0.0280 - val_loss: 4.0628 - val_categorical_accuracy: 0.0000e+00
Epoch 52/1000
6/6 [==============================] - 1s 131ms/step - loss: 3.9004 - categorical_accuracy: 0.0249 - val_loss: 3.9938 - val_categorical_accuracy: 0.0000e+00
Epoch 53/1000
6/6 [==============================] - 1s 130ms/step - loss: 3.8908 - categorical_accuracy: 0.0233 - val_loss: 4.1276 - val_categorical_accuracy: 0.0248
Epoch 54/1000
6/6 [==============================] - 1s 133ms/step - loss: 3.8924 - categorical_accuracy: 0.0311 - val_loss: 4.1217 - val_categorical_accuracy: 0.0

6/6 [==============================] - 1s 134ms/step - loss: 3.8679 - categorical_accuracy: 0.0327 - val_loss: 4.2071 - val_categorical_accuracy: 0.0000e+00
Epoch 98/1000
6/6 [==============================] - 1s 133ms/step - loss: 3.8656 - categorical_accuracy: 0.0327 - val_loss: 4.2820 - val_categorical_accuracy: 0.0000e+00
Epoch 99/1000
6/6 [==============================] - 1s 134ms/step - loss: 3.8634 - categorical_accuracy: 0.0358 - val_loss: 4.2756 - val_categorical_accuracy: 0.0000e+00
Epoch 100/1000
6/6 [==============================] - 1s 132ms/step - loss: 3.8672 - categorical_accuracy: 0.0202 - val_loss: 4.2000 - val_categorical_accuracy: 0.0000e+00
Epoch 101/1000
6/6 [==============================] - 1s 131ms/step - loss: 3.8672 - categorical_accuracy: 0.0311 - val_loss: 4.2970 - val_categorical_accuracy: 0.0000e+00
Epoch 102/1000
6/6 [==============================] - 1s 131ms/step - loss: 3.8636 - categorical_accuracy: 0.0327 - val_loss: 4.2849 - val_categorical_accura

Epoch 145/1000
6/6 [==============================] - 1s 133ms/step - loss: 3.8488 - categorical_accuracy: 0.0187 - val_loss: 4.5031 - val_categorical_accuracy: 0.0000e+00
Epoch 146/1000
6/6 [==============================] - 1s 134ms/step - loss: 3.8329 - categorical_accuracy: 0.0280 - val_loss: 4.2517 - val_categorical_accuracy: 0.0000e+00
Epoch 147/1000
6/6 [==============================] - 1s 134ms/step - loss: 3.8743 - categorical_accuracy: 0.0171 - val_loss: 4.5105 - val_categorical_accuracy: 0.0000e+00
Epoch 148/1000
6/6 [==============================] - 1s 134ms/step - loss: 3.8283 - categorical_accuracy: 0.0358 - val_loss: 4.5074 - val_categorical_accuracy: 0.0000e+00
Epoch 149/1000
6/6 [==============================] - 1s 131ms/step - loss: 3.8321 - categorical_accuracy: 0.0249 - val_loss: 4.5145 - val_categorical_accuracy: 0.0000e+00
Epoch 150/1000
6/6 [==============================] - 1s 132ms/step - loss: 3.8338 - categorical_accuracy: 0.0295 - val_loss: 4.5299 - val_c

Epoch 193/1000
6/6 [==============================] - 1s 128ms/step - loss: 3.8159 - categorical_accuracy: 0.0373 - val_loss: 3.8856 - val_categorical_accuracy: 0.1056
Epoch 194/1000
6/6 [==============================] - 1s 130ms/step - loss: 3.8015 - categorical_accuracy: 0.0280 - val_loss: 4.7096 - val_categorical_accuracy: 0.0000e+00
Epoch 195/1000
6/6 [==============================] - 1s 125ms/step - loss: 3.8063 - categorical_accuracy: 0.0311 - val_loss: 4.7149 - val_categorical_accuracy: 0.0000e+00
Epoch 196/1000
6/6 [==============================] - 1s 127ms/step - loss: 3.8126 - categorical_accuracy: 0.0249 - val_loss: 4.7199 - val_categorical_accuracy: 0.0000e+00
Epoch 197/1000
6/6 [==============================] - 1s 130ms/step - loss: 3.8126 - categorical_accuracy: 0.0280 - val_loss: 4.5718 - val_categorical_accuracy: 0.0000e+00
Epoch 198/1000
6/6 [==============================] - 1s 135ms/step - loss: 3.8023 - categorical_accuracy: 0.0327 - val_loss: 4.7148 - val_categ

Epoch 241/1000
6/6 [==============================] - 1s 134ms/step - loss: 3.8161 - categorical_accuracy: 0.0202 - val_loss: 4.9647 - val_categorical_accuracy: 0.0000e+00
Epoch 242/1000
6/6 [==============================] - 1s 134ms/step - loss: 3.7996 - categorical_accuracy: 0.0249 - val_loss: 4.8558 - val_categorical_accuracy: 0.0000e+00
Epoch 243/1000
6/6 [==============================] - 1s 134ms/step - loss: 3.7940 - categorical_accuracy: 0.0264 - val_loss: 4.9817 - val_categorical_accuracy: 0.0000e+00
Epoch 244/1000
6/6 [==============================] - 1s 131ms/step - loss: 3.8086 - categorical_accuracy: 0.0311 - val_loss: 4.9851 - val_categorical_accuracy: 0.0000e+00
Epoch 245/1000
6/6 [==============================] - 1s 137ms/step - loss: 3.7997 - categorical_accuracy: 0.0358 - val_loss: 4.9959 - val_categorical_accuracy: 0.0000e+00
Epoch 246/1000
6/6 [==============================] - 1s 136ms/step - loss: 3.8080 - categorical_accuracy: 0.0171 - val_loss: 4.0887 - val_c

KeyboardInterrupt: 